# NLP 2 - Lab 06
## Adrien Giget, Tanguy Malandain, Denis Stojiljkovic

In [348]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import math
from timeit import default_timer as timer
from tqdm import tqdm

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

import torch.nn.functional as F

from sacrebleu.metrics import BLEU

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [3]:
# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

In [4]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

In [5]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

In [6]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [7]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

In [8]:
# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [9]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [10]:
# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [11]:
# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [12]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [13]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [14]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

In [15]:
# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [16]:
# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

In [17]:
# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [18]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))

In [19]:
def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [20]:
# Transformer training

NUM_EPOCHS = 18

for epoch in tqdm(range(1, NUM_EPOCHS+1)):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Pour gagner du temps, nous avons sauvegarder le modèle dans le fichier `transformer_18_epochs.pt` pour ne pas avoir à relancer à chaque fois l'entraînement du modèle.

In [21]:
# Save the transformer to not rerun the training every time
# torch.save(transformer.state_dict(), 'transformer_18_epochs.pt')

In [22]:
# Load the saved transformer
# transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
#                                  NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
# transformer = transformer.to(DEVICE)
# transformer.load_state_dict(torch.load('transformer_18_epochs.pt'))

<All keys matched successfully>

In [23]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

In [231]:
# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str, decode_function, **decode_args):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = decode_function(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, **decode_args).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [25]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu .", greedy_decode))

 A group of people stand in front of an igloo 


Si l'on traduit avec Google Translate, on a: "A group of people stand in front of an igloo".

## (4 points) Theoretical questions


### In the positional encoding, why are we using a combination of sinus and cosinus?


Les encodages positionnels sont utilisés pour donner une idée de l'ordre des mots, étant donné que le mécanisme d'auto-attention du transformers ne comprend pas intrinsèquement les informations positionnelles ou séquentielles. Ces codages sont ajoutés aux enchâssements de mots avant leur entrée dans le transformer.

Les fonctions sinus et cosinus sont utilisées parce qu'elles permettent d'encoder la position avec une combinaison unique et réversible de valeurs qui peuvent s'adapter facilement à différentes longueurs de séquences. Chaque dimension du codage positionnel correspond à une fonction sinus ou cosinus avec une fréquence différente, ce qui permet au modèle d'apprendre à assister aux positions relatives puisque pour tout décalage fixe k, `Pos + k` peut être représenté comme une fonction linéaire de Pos.

Ainsi, l'utilisation de ces fonctions permet au modèle d'extrapoler à des longueurs de séquence en dehors de l'ensemble d'apprentissage.

### In the Seq2SeqTransformer class:
- What is the parameter nhead for?
- What is the point of the generator?

Le paramètre nhead de la classe Seq2SeqTransformer fait référence au nombre de têtes dans le mécanisme d'auto-attention multi-têtes. Dans ce mécanisme, le modèle crée plusieurs sous-espaces (ou "têtes") des données d'entrée et applique à chacun d'eux un dot-product mis à l'échelle. Les résultats sont ensuite concaténés et transformés linéairement pour obtenir la sortie finale. L'idée est que différentes têtes peuvent apprendre à se concentrer sur différents types d'informations dans les données d'entrée, ce qui rend le modèle plus puissant et plus flexible. Le paramètre nhead détermine le nombre de ces sous-espaces.

Dans le contexte d'un modèle de transformer, un générateur est généralement une couche linéaire qui fait correspondre la sortie du transformer (qui est dans la dimension de l'état caché) à la taille du vocabulaire. L'objectif est de générer les jetons de sortie finaux à partir de la représentation interne du modèle. Dans de nombreux cas, après le générateur, une fonction softmax est appliquée pour obtenir une distribution de probabilité sur tous les mots de sortie possibles.

### Describe the goal of the create_mask function. Why does it handle differently the source and target masks?


La fonction `create_mask` génère des masques pour un modèle séquence à séquence, en particulier pour les séquences source (`src`) et cible (`tgt`).

1. Le masque `src_mask` permet au modèle d'accéder à tous les tokens de la source car la séquence d'entrée entière est généralement accessible pendant le décodage.
2. `tgt_mask` empêche le "look-ahead", en interdisant l'accès aux futurs tokens cibles pendant la prédiction en cours.
3. `src_padding_mask` et `tgt_padding_mask` ignorent les jetons de remplissage ajoutés pour normaliser la longueur des séquences.

La différence de traitement est due à la nature séquentielle de la tâche. Pour les sources, tous les tokens sont accessibles en même temps ; pour les cibles, le modèle doit prédire tokens par tokens sans voir les tokens futurs.

## (6 points) Decoding functions

The tutorial uses a greedy approach at decoding. Implement the following variations.
* (3 points) A top-k sampling with temperature.

In [26]:
def top_k_sampling_decode(model, src, src_mask, max_len, start_symbol, temperature=1.0, top_k=10):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        logits = model.generator(out[:, -1])
        
        # Apply temperature
        logits = logits / temperature
        
        # Extract top-k tokens
        probs = F.softmax(logits, dim=-1)
        top_k_probs, top_k_indices = torch.topk(probs, top_k)
        
        # Sample from top-k probabilities
        next_word = torch.multinomial(top_k_probs, num_samples=1)
        next_word_idx = top_k_indices.gather(dim=1, index=next_word)
        next_word_idx = next_word_idx.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word_idx)], dim=0)
        if next_word_idx == EOS_IDX:
            break
    return ys

In [84]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu.", top_k_sampling_decode, temperature=0.8, top_k=10))

 A group of people stand in front of an igloo 


* (1 point) A top-p sampling with temperature.

In [322]:
def top_p_sampling_decode(model, src, src_mask, max_len, start_symbol, temperature=1.0, top_p=0.9):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        logits = model.generator(out[:, -1])

        # Apply temperature
        logits = logits / temperature

        # Convert logits to probabilities
        probs = F.softmax(logits, dim=-1)

        # Sort probabilities and corresponding indices
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)

        # Compute cumulative probabilities
        cum_probs = torch.cumsum(sorted_probs, dim=-1)

        # Create mask for probabilities that exceed the threshold
        mask = cum_probs > top_p

        # If at least one token was masked, ignore the first one (it has the highest probability)
        if mask.sum() > 0:
            if mask.shape[0] > 1:
                mask = torch.cat((torch.tensor([False]).to(DEVICE), mask[:-1]))
            else:
                mask = torch.tensor([False]).to(DEVICE)
        
        # Apply the mask to sorted probabilities and indices
        filtered_probs = sorted_probs.masked_fill(mask, 0.0)
        filtered_indices = sorted_indices.masked_fill(mask, 0)

        # Sample from the filtered probabilities
        next_word_idx = torch.multinomial(filtered_probs, num_samples=1)
        next_word_idx = filtered_indices.gather(dim=1, index=next_word_idx)
        next_word_idx = next_word_idx.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word_idx)], dim=0)
        if next_word_idx == EOS_IDX:
            break
    return ys

In [321]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu .", top_p_sampling_decode, temperature=0.4, top_p=0.9))

 A group of people standing in front of an igloo 


* (2 point) Play with the k, p and temperature parameters, and qualitatively compare a few (at least 3) translation samples for each approach (even the greedy one).

On va d'abord récupérer 3 sample à traduire ainsi que leur target.

In [101]:
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

i = 0
phrases, targets = [], []

for src, tgt in val_iter:
    phrases.append(src)
    targets.append(tgt)
    
    i += 1
    if i == 3:
        break
        
phrase1, target1 = phrases[0], targets[0]
phrase2, target2 = phrases[1], targets[1]
phrase3, target3 = phrases[2], targets[2]

In [104]:
print(f"Premer sample: {phrase1}\nTarget: {target1}")

Premer sample: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
Target: A group of men are loading cotton onto a truck


In [110]:
print(f"Deuxième sample: {phrase2}\nTarget: {target2}")

Deuxième sample: Ein Mann schläft in einem grünen Raum auf einem Sofa.
Target: A man sleeping in a green room on a couch.


In [111]:
print(f"Troisième sample: {phrase3}\nTarget: {target3}")

Troisième sample: Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.
Target: A boy wearing headphones sits on a woman's shoulders.


### Greedy Approach

In [138]:
print(translate(transformer, phrase1, greedy_decode), "Target:", target1)

 A group of men loading animal cotton into a truck .  Target: A group of men are loading cotton onto a truck


In [123]:
print(translate(transformer, phrase2, greedy_decode), "Target:", target2)

 A man is sleeping on a couch in a green room .  Target: A man sleeping in a green room on a couch.


In [124]:
print(translate(transformer, phrase3, greedy_decode), "Target:", target3)

 A boy with headphones sitting on his shoulders while sitting on a woman .  Target: A boy wearing headphones sits on a woman's shoulders.


Les traductions générées par le modèle semblent être globalement correctes, mais elles ne sont pas parfaites. Elles parviennent à transmettre le sens général de chaque phrase, mais il y a des erreurs spécifiques dans chaque exemple.

"A group of men loading animal cotton into a truck." Au lieu de "animal cotton", le terme correct serait simplement "cotton". Il semble que le modèle ait mal traduit ou ajouté des informations supplémentaires qui n'étaient pas dans la phrase d'origine.

"A man is sleeping on a couch in a green room." Cette traduction semble correcte.

"A boy with headphones sitting on his shoulders while sitting on a woman." Ici, le modèle a mal interprété la relation spatiale entre le garçon et la femme.

### Top_k Approach

### Phrase 1: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen

In [167]:
print(translate(transformer, phrase1, top_k_sampling_decode, temperature=0.8, top_k=10))

 A group of men loading cotton into a truck . 


In [168]:
print(translate(transformer, phrase1, top_k_sampling_decode, temperature=0.8, top_k=15))

 A group of men unloading onto a truck . 


In [169]:
print(translate(transformer, phrase1, top_k_sampling_decode, temperature=0.4, top_k=10))

 A group of men loading animal cotton over a truck . 


Pour le top_k_sampling_decode, le résultat ne sera pas forcément le même, on va donc prendre en compte une simulation que l'on a eu:

**Target : "A group of men are loading cotton onto a truck"**

**Température=0.8, top_k=10** : "A group of men loading cotton into a truck."<br>Cette traduction est correcte et assez proche de la phrase cible.

**Température=0.8, top_k=15** : "A group of men unloading onto a truck ."<br>Cette traduction a une erreur majeure : "unloading" au lieu de "loading". De plus, le "cotton" a été omis.

**Température=0.4, top_k=10** : "A group of men loading animal cotton over a truck ."<br>Ici aussi, nous avons une erreur où "animal cotton" est incorrect et "over" n'est pas la bonne préposition dans ce contexte.

### Phrase 2: Ein Mann schläft in einem grünen Raum auf einem Sofa.

In [170]:
print(translate(transformer, phrase2, top_k_sampling_decode, temperature=0.8, top_k=10))

 A man sleeping in a green room on a couch . 


In [171]:
print(translate(transformer, phrase2, top_k_sampling_decode, temperature=0.8, top_k=15))

 A man is asleep on a couch in a green room . 


In [172]:
print(translate(transformer, phrase2, top_k_sampling_decode, temperature=0.4, top_k=10))

 A man is sleeping in a green room on a couch . 


**Target : "A man sleeping in a green room on a couch."**

Les trois traductions pour cette phrase sont toutes assez correctes, bien que certaines varient légèrement en termes de structure grammaticale.

### Phrase 3: Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.

In [192]:
print(translate(transformer, phrase3, top_k_sampling_decode, temperature=0.8, top_k=10))

 A boy wearing headphones is sitting on his shoulders by a woman . 


In [208]:
print(translate(transformer, phrase3, top_k_sampling_decode, temperature=0.8, top_k=15))

 A young boy with headphones on sitting on top of a woman . 


In [333]:
print(translate(transformer, phrase3, top_k_sampling_decode, temperature=0.4, top_k=10))

 A boy with headphones sitting on his shoulders while sitting on a woman . 


**Target "A boy wearing headphones sits on a woman's shoulders."**

**Température=0.8, top_k=10** : "A boy with headphones sitting on his shoulders with a woman on her shoulders ."<br>Cette traduction est incorrecte car le garçon est censé être assis sur les épaules de la femme, pas les siennes.

**Température=0.8, top_k=15** : "A young boy wearing headphones is sitting on her shoulders ."<br>Cette phrase est partiellement correcte, mais le pronom "her" est ambigu sans contexte supplémentaire.

**Température=0.4, top_k=10** : "A boy wearing headphones is sitting on his shoulders with a woman ."<br>Ici aussi, la phrase suggère de façon incorrecte que le garçon est assis sur ses propres épaules.

Les traductions générées par le modèle en utilisant le top_k_sampling_decode présentent des erreurs et des variations intéressantes. L'utilisation de la méthode de top_k_sampling_decode permet d'introduire plus de diversité dans les traductions générées par le modèle. Cependant, cela peut aussi introduire des erreurs ou des phrases qui n'ont pas de sens. Une valeur plus élevée du top_k conduit à plus de diversité mais peut aussi conduire à plus d'erreurs et une valeur plus élevée de la température donne une traduction qui varie plus entre les différentes tentatives de traduction.

### Top_p Approach

### Phrase 1: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen

In [337]:
print(translate(transformer, phrase1, top_p_sampling_decode, temperature=0.8, top_p=0.8))

 A group of men putting pot on a truck into a truck . 


In [338]:
print(translate(transformer, phrase1, top_p_sampling_decode, temperature=0.8, top_p=0.5))

 A group of men putting out boxes on a truck . 


In [339]:
print(translate(transformer, phrase1, top_p_sampling_decode, temperature=0.4, top_p=0.8))

 A group of men loading animal cotton into a truck . 


Pour le top_k_sampling_decode, le résultat ne sera pas forcément le même, on va donc prendre en compte une simulation que l'on a eu:

**Target : "A group of men are loading cotton onto a truck"**

**Température=0.8, top_k=0.8** : "A group of men putting pot on a truck into a truck ."<br>La phrase est mal traduite, les mots 'putting pot' ne correspondent pas à l'original et il y a une répétition.

**Température=0.8, top_k=0.5** : "A group of men putting out boxes on a truck ."<br>La traduction est incorrecte, 'putting out boxes on a truck' ne contient pas l'information 'cotton' et 'on' n'est pas la bonne préposition.

**Température=0.4, top_k=0.8** : "A group of men loading animal cotton into a truck ."<br>La phrase est correcte mais on a l'ajout de 'animal' qui est en trop.

### Phrase 2: Ein Mann schläft in einem grünen Raum auf einem Sofa.

In [340]:
print(translate(transformer, phrase2, top_p_sampling_decode, temperature=0.8, top_p=0.8))

 A man is sleeping in a green room on a sofa . 


In [341]:
print(translate(transformer, phrase2, top_p_sampling_decode, temperature=0.8, top_p=0.5))

 A man is sleeping while on a couch in a green room . 


In [342]:
print(translate(transformer, phrase2, top_p_sampling_decode, temperature=0.4, top_p=0.8))

 A man is sleeping on a couch in a green room . 


**Target : "A man sleeping in a green room on a couch."**

Les trois traductions pour cette phrase sont toutes correctes, même si pour la première 'couch' a été remplacé par 'sofa'.

### Phrase 3: Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.

In [343]:
print(translate(transformer, phrase3, top_p_sampling_decode, temperature=0.8, top_p=0.8))

 A boy wearing sunglasses is sitting on a woman 's shoulders . 


In [344]:
print(translate(transformer, phrase3, top_p_sampling_decode, temperature=0.8, top_p=0.5))

 A boy wearing headphones is sitting on a woman 's shoulders . 


In [345]:
print(translate(transformer, phrase3, top_p_sampling_decode, temperature=0.4, top_p=0.8))

 A boy wearing headphones is sitting on his shoulders while sitting on a woman . 


**Target "A boy wearing headphones sits on a woman's shoulders."**

**Température=0.8, top_k=0.8** : "A boy wearing sunglasses is sitting on a woman 's shoulders ."<br>Une traduction assez correcte mais qui a cependant remplacé 'headphones' par 'sunglasses'.

**Température=0.8, top_k=0.5** : "A boy wearing headphones is sitting on a woman 's shoulders ."<br>La traduction est correcte.

**Température=0.4, top_k=0.8** : "A boy wearing headphones is sitting on his shoulders while sitting on a woman ."<br>Ici 'shoulders' a été associé au garçon et non pas à la femme.

De la même façon que le top_k_sampling_decode, les résultats ne sont pas toujours précis et dépendent fortement des valeurs de température et de top_p. Le top_p contrôle la diversité des prédictions. Un top_p plus faible signifie que seuls les tokens les plus probables sont conservés, tandis qu'un top_p plus élevé permet de conserver plus de tokens.

## **(2 points)** Compute the BLEU score of the model

Use the [sacreBLEU](https://github.com/mjpost/sacreBLEU) implementation to evaluate your model and quantitatively compare the 3 implemented decoding approaches on the test set. Explain what all the output values mean (when using the `corpus_score` function).

In the [python section](https://github.com/mjpost/sacrebleu#using-sacrebleu-from-python), you'll notice the library accepts more than just one possible translation as reference, but the given dataset only has one translation per sample.

In [379]:
def evaluate_model(model, decode_function, **decode_args):
    references = []
    candidates = []
    
    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    
    for src_sentence, tgt_sentence in val_iter:
        translation = translate(model, src_sentence, decode_function, **decode_args)
        candidates.append(translation)
        references.append([tgt_sentence])
        
    bleu = BLEU()
    
    score = bleu.corpus_score(candidates, references)
    return score

In [351]:
# Evaluate with greedy decode
greedy_bleu = evaluate_model(transformer, greedy_decode)
print('Greedy Decode BLEU:', greedy_bleu)

Greedy Decode BLEU: BLEU = 35.93 100.0/60.0/22.2/12.5 (BP = 1.000 ratio = 1.000 hyp_len = 11 ref_len = 11)


Expliquons les outputs de la fonction `corpus_score` sur le greedy_decode:

- BLEU = 35.93 : c'est le score BLEU pour le texte évalué qui mesure la qualité de la sortie de la traduction automatique par rapport aux traductions de référence.

- 100.0/60.0/22.2/12.5 : Ce sont les scores de précision pour les correspondances 1-gramme, 2-grammes, 3-grammes et 4-grammes respectivement. Les scores mesurent combien des n-grammes dans les traductions générées par la machine correspondent aux n-grammes dans les traductions de référence.

- BP = 1.000 : Il s'agit de la Pénalité de Brèveté. La Pénalité de Brèveté pénalise les traductions automatiques qui sont plus courtes que les traductions de référence, car les traductions plus courtes omettent souvent des informations importantes. La Pénalité de Brèveté est un nombre compris entre 0 et 1. Si la traduction automatique est de la même longueur ou plus longue que la référence, la Pénalité de Brèveté est 1 et n'a aucun effet.

- ratio = 1.000 : Il s'agit du rapport de longueur entre la traduction automatique et la traduction de référence.

- hyp_len = 11 : C'est la longueur de l'hypothèse (traduction automatique) en tokens.

- ref_len = 11 : C'est la longueur de référence effective.

In [352]:
# Evaluate with top-k sampling decode
top_k_bleu = evaluate_model(transformer, top_k_sampling_decode, temperature=0.8, top_k=10)
print('Top-k Decode BLEU:', top_k_bleu)

Top-k Decode BLEU: BLEU = 40.82 100.0/77.8/25.0/14.3 (BP = 1.000 ratio = 1.000 hyp_len = 10 ref_len = 10)


In [353]:
# Evaluate with top-p sampling decode
top_p_bleu = evaluate_model(transformer, top_p_sampling_decode, temperature=0.8, top_p=0.8)
print('Top-p Decode BLEU:', top_p_bleu)

Top-p Decode BLEU: BLEU = 35.08 90.9/60.0/22.2/12.5 (BP = 1.000 ratio = 1.000 hyp_len = 11 ref_len = 11)


Dans notre cas, le top_k_sampling_decode semble offrir la meilleure performance avec un score BLEU de 40.82. Elle présente également une précision supérieure pour les 2-grammes, 3-grammes et 4-grammes par rapport aux autres approches. Cependant, pour être certain de ce résultat, il faudrait faire varier les hyperparamètres temperature, top_k et top_p.

Using the `translate` function provided in the tutorial is pretty slow, as it translate text by text. It's recommended you modify the function to accept a list of texts as input, and batch them for translations (also **bonus point**).

In [354]:
def translate(model: torch.nn.Module, src_sentences: list, decode_function, **decode_args):
    model.eval()
    translated_sentences = []
    for src_sentence in src_sentences:
        src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = decode_function(
            model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, **decode_args).flatten()
        translated_sentence = " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy())))
        translated_sentence = translated_sentence.replace("<bos>", "").replace("<eos>", "")
        translated_sentences.append(translated_sentence)
    return translated_sentences

**\[Bonus\]** Use part of the test set to perform an hyperparameters search on the value of temperature, k, and p. Note that, normally, this should be done on a validation set, not the test set.

Malheureusement, nous n'avons pas réussi à batcher les phrases avant de les passer dans la fonction translate, par conséquent nous n'avons pas eu le temps de réaliser la recherche des hyperparamètres.

In [ ]:
temperature_values = [0.5, 0.7, 0.9, 1.0, 1.2]
top_k_values = [5, 10, 20, 40, 50]

best_score = 0.0
best_params = None

for temperature in tqdm(temperature_values):
    for top_k in top_k_values:
        # Compute BLEU score
        current_score = evaluate_model(transformer, top_k_sampling_decode, temperature=temperature, top_k=top_k).score
        if current_score > best_score:
            best_score = current_score
            best_params = (temperature, top_k)
            
print(f"Best score: {best_score}")
print(f"Best parameters: Temperature={best_params[0]}, Top-k={best_params[1]}")

In [ ]:
top_p_values = [0.5, 0.7, 0.9, 1.0]

best_score = 0.0
best_params = None

for temperature in tqdm(temperature_values):
    for top_p in top_p_values:
        # Compute BLEU score
        current_score = evaluate_model(transformer, top_p_sampling_decode, temperature=temperature, top_p=top_p).score
        if current_score > best_score:
            best_score = current_score
            best_params = (temperature, top_p)
                
print(f"Best score: {best_score}")
print(f"Best parameters: Temperature={best_params[0]}, Top-p={best_params[1]}")